# Reflection

## Understanding Reflection in AI Agents
### What is Reflection?
Reflection is the ability of an AI agent to:
- Monitor its own actions and decisions
- Evaluate its performance
- Learn from experience
- Adjust its behavior based on feedback
- Improve over time

Think of reflection like how a student might review their homework after completing it, or how a chef tastes their dish while cooking to make adjustments.

### 1. Let's start with basic imports

In [1]:
import os
from openai import OpenAI
import json

# initialize MaLLaM client
mallam_api_key = os.getenv("MALLAM_API_KEY")

client = OpenAI(
    base_url="https://api.mesolitica.com",
    api_key=mallam_api_key
)

### 2. Create a Simple Reflection Agent
First, let's create a basic agent that can reflect on its responses

In [14]:
# create a function that will do a reflection on responses
def simple_reflection(response_to_analyze):
    """A simple function that reflects on a given response and suggests improvements"""
    
    reflection_prompt = [
        {
            "role": "system",
            "content": """You are a reflective agent. 
            Analyze the given response and provide:
            1. What was good about the response?
            2. What could be improved?
            3. Suggestions for next time
            Keep your analysis simple and clear. Reply in English."""
        },
        {
            "role": "user",
            "content": f"Please analyze this response: {response_to_analyze}"
        }
    ]
    
    reflection = client.chat.completions.create(
        model="mallam-small",
        messages=reflection_prompt,
        max_tokens=512
    )
    
    return reflection.choices[0].message.content

### 3. Create a Basic Agent with Reflection
Now let's create an agent that can respond and reflect

In [15]:
def agent_with_reflection(user_input):
    """
    A basic agent that:
    1. Generate a response
    2. Reflects on its response
    3. Returns both the response and reflection
    """
    
    response = client.chat.completions.create(
        model="mallam-small",
        messages=[
            {"role": "user", "content": f"Reply in English and keep it short and concise. Input: {user_input}"}
        ]
    )
    
    initial_response = response.choices[0].message.content
    
    # reflect on the response
    reflection = simple_reflection(initial_response)
    
    return {
        "response": initial_response,
        "reflection": reflection
    }

### 4. Let's test our Basic Reflection Agent

In [16]:
# Test with simple question
print("Test 1: Simple Question")
result = agent_with_reflection(
    "What is machine learning?"
)
print("\nResponse:", result["response"])
print("\nReflection:", result["reflection"])

Test 1: Simple Question

Response: Machine learning is a subset of artificial intelligence that focuses on developing algorithms and statistical models that enable computers to learn from data without being explicitly programmed. It involves training models on large amounts of data and using them to make predictions or decisions based on new data.

Reflection: **What was good about the response?**

The response provides a concise definition of machine learning, emphasizing its role within artificial intelligence and highlighting the key aspects of developing algorithms and statistical models. It also correctly mentions the process of training models on data to make predictions or decisions.

**What could be improved?**

While the response is accurate, it could benefit from more depth and clarity. For example, it doesn't specify different types of machine learning (supervised, unsupervised, reinforcement learning), nor does it mention any real-world applications or examples. Additionall

In [17]:
# Test with a more complex request
print("Test 2: Complex Question")
result = agent_with_reflection(
    "Can you explain the differences between supervised and unsupervised learning, and give examples of each?"
)
print("\nResponse:", result["response"])
print("\nReflection:", result["reflection"])

Test 2: Complex Question

Response: Sure! In machine learning, there are two main types of learning methods: supervised and unsupervised.

In supervised learning, an algorithm is trained on a labeled dataset, meaning that for each input data point, there's a corresponding output label. The goal is to learn a mapping from inputs to outputs. For example, consider a spam filter that classifies emails as either spam or not spam based on features like email content and sender information. Here, the input is the email data, and the output is the classification label (spam or not spam). Another example is predicting house prices based on features such as size, location, and number of bedrooms, where the input is the features and the output is the price.

On the other hand, unsupervised learning deals with unlabeled datasets, meaning there are no output labels provided for the algorithm to learn from. Instead, the algorithm tries to find hidden patterns or intrinsic structures in the data. Clu

### 5. Adding Basic Improvement Mechanism
Let's add the ability to improve the responses based on reflection

In [23]:
def improved_response(user_input, reflection):
    """Generate an improved response based on reflection"""
    improvement_prompt = [
        {
            "role": "system",
            "content": """You are an AI assisstant. You have the ability to do reflection.
            Generate an improved response based on the original prompt and reflection feedback."""
        },
        {
            "role": "user",
            "content": f"""Original prompt: {user_input}
            Reflection feedback: {reflection}
            Please provide an improved response."""
        }
    ]
    
    response = client.chat.completions.create(
        model="mallam-small",
        messages=improvement_prompt,
        max_tokens=1024
    )
    
    return response.choices[0].message.content

In [24]:
def improved_reflective_agent(user_input):
    """An Agent that:
    1. Generates initial response
    2. Reflects on it
    3. Generates improved response
    4. Returns all steps
    """
    
    # get initial response and reflection
    first_response = agent_with_reflection(user_input)
    
    # generate improved response
    reflective_response = improved_response(user_input, first_response["reflection"])
    
    return {
        "initial_response": first_response["response"],
        "reflection": first_response["reflection"],
        "improved_response": reflective_response
    }

### 6. Test the Improved Agent

In [25]:
print("Test with Improvement:")
result = improved_reflective_agent(
    "What are the main components of an AI system?"
)
print("\nInitial Response:", result["initial_response"])
print("\nReflection:", result["reflection"])
print("\nImproved Response:", result["improved_response"])

Test with Improvement:

Initial Response: Main components of an AI system include data collection, data preprocessing, feature extraction, model selection, model training, model evaluation, and deployment.

Reflection: 1. **What was good about the response?**
   - The response accurately identifies key components of an AI system.
   - It provides a concise overview that touches on essential steps in the AI development process.

2. **What could be improved?**
   - The response is somewhat vague and lacks depth in explaining each component.
   - It does not mention the iterative nature of AI development or the importance of validation and testing phases.
   - Additionally, it might benefit from including real-world examples or specific tools and techniques used in each phase.

3. **Suggestions for next time**
   - Provide more detailed explanations for each component.
   - Discuss the iterative process and how different stages may loop back to earlier steps based on feedback and performa